<span style="color:lightgreen; font-size:30px">**PG102 - Análisis de datos en Geología**</span>
***
<span style="color:gold; font-size:30px">**Análisis descriptivo**</span>
***

<span style="font-size:20px"> **Autor: Kevin Alexander Gómez** </span>

<span style="font-size:16px"> **Contacto: kevinalexandr19@gmail.com | [Linkedin](https://www.linkedin.com/in/kevin-alexander-g%C3%B3mez-2b0263111/) | [Github](https://github.com/kevinalexandr19)** </span>
***

Bienvenido al curso PG102 - Análisis de datos en Geología!!!

Vamos a revisar ejemplos de <span style="color:gold">análisis de datos</span> en Geología usando código en Python.\
Es necesario que tengas un conocimiento previo en programación con Python, estadística y geología general.

<span style="color:lightgreen"> Este notebook es parte del proyecto [**Python para Geólogos**](https://github.com/kevinalexandr19/manual-python-geologia), y ha sido creado con la finalidad de facilitar el aprendizaje en Python para estudiantes y profesionales en el campo de la Geología. </span>

En el siguiente índice, encontrarás los temas que componen este notebook:

<span style="font-size:20px"> **Índice** </span>
***
- [¿Qué es el análisis descriptivo?](#parte-1)
- [Procesamiento de datos](#parte-2)
- [Caso de estudio: ocurrencia de terremotos cercanos a Lima](#parte-3)
- [Visualización 3D con Plotly](#parte-4)

***

Antes de empezar tu camino en programación geológica...\
Recuerda que puedes ejecutar un bloque de código usando `Shift` + `Enter`:

In [ ]:
2 + 2

Si por error haces doble clic sobre un bloque de texto (como el que estás leyendo ahora mismo), puedes arreglarlo usando también `Shift` + `Enter`.
***

<a id="parte-1"></a>

### <span style="color:lightgreen">**¿Qué es el análisis descriptivo?**</span>
***

De acuerdo con el [Glosario IT de Gartner](https://www.gartner.com/en/information-technology/glossary/descriptive-analytics#:~:text=Descriptive%20Analytics%20is%20the%20examination,%2C%20tables%2C%20or%20generated%20narratives.), el análisis descriptivo es la primera fase del **análisis de datos**.\
Consiste en **examinar** los datos con el fin de responder a la pregunta **¿qué ha sucedido?** o **¿qué está sucediendo?**.

En esta fase, los datos generados en tiempo real e históricos se analizan a través de métodos estadísticos simples y técnicas de visualización.

<img src="resources/analytic_value_escalator.jpg" alt="Las 4 fases en el análisis de datos" width="700"/>

> Es común reportar los resultados de esta fase a través de reportes, resúmenes o dashboards.

Antes de empezar con el código, mencionaremos algunas de las ventajas de usar Python en el análisis de datos:
    
- Aprender a programar te permite desarrollar tus habilidades analíticas y de resolución de problemas.
- Puedes crear e implementar diferentes herramientas de manera independiente y usarlas en cada etapa del análisis de datos.
- <span style="color:gold">Python</span> es uno de los lenguajes más usados en <span style="color:lightgreen">Ciencia de Datos</span> debido a su sintáxis sencilla y clara escritura.
- Todos los trabajos realizados en <span style="color:gold">Python</span> son flexibles y escalables.
***

<a id="parte-2"></a>

### <span style="color:lightgreen">**Procesamiento de datos**</span>
***

Usaremos un archivo Excel perteneciente a la base de datos del **IGP (Instituto Geofísico del Perú)**.\
Específicamente, la información que contiene describe la *ocurrencia de terremotos en Perú desde 1960 hasta 2021*.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="ticks", context="talk") # Mejor calidad visual
plt.style.use("dark_background") # Gráficos con fondo oscuro

Empezaremos usando la función `read_excel` para leer el archivo ubicado en la carpeta `files`.\
Asignaremos la información a un DataFrame de nombre `data`.

In [ ]:
data = pd.read_excel("files/sismos.xlsx")

El tamaño de filas y columnas en la tabla se puede observar usando el atributo `shape`:

In [ ]:
data.shape

La tabla contiene 6 columnas y casi 19,000 filas.

Observamos las primeras 5 filas de `data` usando el método `head`:

In [ ]:
data.head()

Mostramos el tipo de datos de cada columna usando el atributo `dtypes`:

In [ ]:
data.dtypes

Notaremos que la tabla contiene los siguientes datos:

- `fecha UTC` : fecha de ocurrencia del terremoto, en **string**.
- `hora UTC` : hora de ocurrencia del terremoto, en **string**.
- `latitud (º)` : latitud del epicentro, en **float**.
- `longitud (º)` : longitud del epicentro, en **float**.
- `profundidad (km)` : profundidad del epicentro, en **integer**.
- `magnitud (M)` : magnitud del terremoto, en **float**.

No usaremos la columna de hora, así que la eliminaremos usando el método `drop`.\
Activaremos el parámetro `inplace=True` para que los cambios se guarden en la tabla.

In [ ]:
data.drop(columns=["hora UTC"], inplace=True)

In [ ]:
data.head()

Ahora, renombraremos las columnas para facilitar su uso dentro del código.\
Primero, crearemos un diccionario que contenga los nombres de las columnas y los nombres que usaremos para reemplazarlos:

In [ ]:
cols = dict(zip(list(data.columns), ["Fecha", "Latitud", "Longitud", "Profundidad", "Magnitud"]))
cols

Ahora, usaremos el método `rename`:

In [ ]:
data.rename(columns=cols, inplace=True)

In [ ]:
data.head()

Ahora, transformaremos el dato de las fechas de **string** a **datetime**:
> **Nota: datetime** es un objeto en Python que representa una fecha.

Seleccionamos la columna con las fechas:

In [ ]:
fecha = data["Fecha"]

In [ ]:
fecha.head(5)

Y usaremos la función `to_datetime` para transformarlo en **datetime**:

In [ ]:
data["Fecha"] = pd.to_datetime(fecha, format="%d/%m/%Y")

Verificamos:

In [ ]:
data.head()

In [ ]:
data.dtypes

<a id="parte-3"></a>

### <span style="color:lightgreen">**Caso de estudio: ocurrencia de terremotos cercanos a Lima**</span>

Vamos a analizar los terremotos ocurridos cerca a Lima, en un radio de 150 kilómetros.\
Empezaremos usando la librería `geopy` para seleccionar aquellos epicentros que se encuentren dentro del radio mencionado:

In [ ]:
from geopy.distance import geodesic

Empezaremos asignando un punto de referencia para la ciudad de Lima:

In [ ]:
punto_lima = [-12.045976, -77.030555]

Seleccionamos las columnas de latitud y longitud de cada epicentro y calcumos su distancia geodésica (en km) al punto de Lima:

In [ ]:
# Distancia geodésica de los epicentros a Lima
filtro = data[["Latitud", "Longitud"]].apply(lambda punto: geodesic(punto_lima, punto).km, axis=1)
filtro.head()

Si establecemos un valor de 150 km como radio, podemos extraer los epicentros que estamos buscando:

In [ ]:
filtro <= 150

Vamos a guardar la tabla filtrada en una variable llamada `Lima` y crearemos una copia para no alterar el original:

In [ ]:
lima = data[filtro <= 150].reset_index(drop=True).copy()

Observamos la nueva tabla:

In [ ]:
lima.head()

También podemos ver un resumen estadístico de la tabla usando el método `describe`:

In [ ]:
lima.describe()

Y mostramos el registro de sismos cercanos a Lima:

In [ ]:
print(f"Se registraron {len(lima)} sismos en un radio de 150 km en torno a Lima.")

Vamos a crear un gráfico que muestre la distribución de los epicentros en torno a la ciudad de Lima.
> Colocaremos 3 puntos adicionales en el gráfico: Ancón, Cañete y Huacho.\
> Crearemos un diccionario que contenga el nombre de estas ciudades y sus respectivas coordenadas.

In [ ]:
punto_ancon = [-11.773241, -77.176009]
punto_cañete = [-13.077772, -76.387440]
punto_huacho = [-11.108526, -77.610365]

puntos = [punto_lima, punto_ancon, punto_cañete, punto_huacho]
nombres = ["Lima", "Ancón", "Cañete", "Huacho"]

In [ ]:
ciudades = dict(zip(nombres, puntos))
ciudades

Verificaremos que los sismos hayan sido seleccionados correctamente (i.e. dentro del radio de 150 km).\
Para esto, crearemos una figura usando la función `scatter`:

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7), subplot_kw={"aspect": 1})
ax.scatter(lima["Longitud"], lima["Latitud"], c="white", s=5)

for ciudad, coordenada in ciudades.items():
    ax.scatter(coordenada[1], coordenada[0], c="r", s=25)
    ax.annotate(ciudad, coordenada[::-1], fontsize=18, c="gold")

ax.set_title("Epicentros alrededor de Lima\n (Radio de 150 km)", fontsize=18)
ax.set_xlabel("Longitud (°)", fontsize=15)
ax.set_ylabel("Latitud (°)", fontsize=15)
ax.grid(linewidth=0.5, alpha=0.5)

plt.show()

Con la información presente, trataremos de responder las siguientes preguntas (usando un gráfico en cada pregunta):

- ¿Cuántos terremotos ocurrieron por año?

- ¿Con qué frecuencia ocurrieron terremotos en cada mes? ¿Qué mes tiene la mayor frecuencia de terremotos?

- ¿Cuál es la distribución de profundidad de los sismos sobre el área de Lima?

***
<span style="color:gold">**¿Cuántos terremotos ocurrieron por año?**</span>

Empezaremos revisando los datos generales:

In [ ]:
lima.head()

Para observar la frecuencia de terremotos que ocurrieron por año, usaremos un **diagrama de barras**.\
Para determinar la **frecuencia de terremotos por año**, tendremos que agrupar los datos de acuerdo al año en que ocurrieron.\
Empezaremos extrayendo el año de ocurrencia de cada terremoto. Usaremos el atributo `year` de la columna `Fecha`:

In [ ]:
lima["Fecha"].apply(lambda fecha: fecha.year)

Y la asignaremos a una nueva columna llamada `Año`:

In [ ]:
lima["Año"] = lima["Fecha"].apply(lambda fecha: fecha.year)

Revisando nuevamente los datos generales:

In [ ]:
lima.head()

Ahora, agruparemos los terremotos de acuerdo al año de ocurrencia.\
Para esto, usaremos el método `groupby` para agrupar los datos y `count` para obtener las frecuencias por año.

In [ ]:
lima.groupby(["Año"])["Año"].count().head(10)

Asignaremos esta información a una variable llamada `sismos`:

In [ ]:
sismos = lima.groupby(["Año"])["Año"].count()

Crearemos una lista con los `años` de ocurrencia, y asignaremos la frecuencia por año en una variable llamada `frecuencia`:

In [ ]:
años = list(sismos.index)
frecuencia = list(sismos.values)

Para terminar, usaremos la función `bar` y los datos en `frecuencia` y `años` para crear el diagrama de barras:

In [ ]:
# Figura principal
fig, ax = plt.subplots(figsize=(12, 5))

# Diagrama de barras
ax.bar(años, frecuencia, color="lightgreen", alpha=0.75, edgecolor="black")

# Agregamos detalles
ax.set_title("Frecuencia de terremotos por año\n Fuente: IGP", fontsize=18)
ax.set_xlabel("Año", fontsize=16)
ax.set_ylabel("Frecuencia", fontsize=16)

# Cambiamos la ubicación de los ticks en el eje X
ax.set_xticks([1965, 1975, 1985, 1995, 2005, 2015])

plt.show()

***
<span style="color:gold">**¿Con qué frecuencia ocurrieron terremotos en cada mes? ¿Qué mes tiene la mayor frecuencia de terremotos?**</span>

Para observar la frecuencia de terremotos por mes, usaremos nuevamente un **diagrama de barras**.\
Para determinar la **frecuencia de terremotos por mes**, tendremos que agrupar los datos de acuerdo al mes en que ocurrieron.\
Empezaremos extrayendo el mes de ocurrencia de cada terremoto. Usaremos el atributo `month` de la columna `Fecha`, y lo asignaremos a una nueva columna llamada `Mes`:

In [ ]:
lima["Mes"] = lima["Fecha"].apply(lambda fecha: fecha.month)

Revisamos los datos generales:

In [ ]:
lima.head()

Ahora, agruparemos los terremotos de acuerdo al mes de ocurrencia.\
Para esto, usaremos el método `groupby` para agrupar los datos y `count` para obtener las frecuencias por mes.\
Asignaremos esta información a una variable llamada `sismos`:

In [ ]:
sismos = lima.groupby(["Mes"])["Mes"].count()

Crearemos una lista con los meses de ocurrencia, y otra lista con las frecuencias:

In [ ]:
mes = list(sismos.index)
frecuencia = list(sismos.values)

Ahora, crearemos el diagrama de barras usando la función `bar` y los datos `frecuencia` y `mes`:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
barra = ax.bar(mes, frecuencia, color="lightgreen", alpha=0.75, edgecolor="black")

ax.set_title("Frecuencia de terremotos por mes\n Fuente: IGP", fontsize=22)

ax.tick_params(left=False, labelleft=False, bottom=False)

nombres_mes = ["Ene", "Feb", "Mar", "Abr", "May", "Jun", "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"]
plt.xticks(ticks=mes, labels=nombres_mes, fontsize=15)

for spine in ax.spines.values():
    spine.set_visible(False)

for bar in barra:
    ax.text(bar.get_x() + bar.get_width()/2,
            bar.get_height() + 1,
            str(int(bar.get_height())), ha="center", fontsize=15)
        
plt.show()

***
<span style="color:gold">**¿Cuál es la distribución de profundidad de los sismos sobre el área de Lima?**</span>

Empezaremos observando la distribución de profundidad a través de un **histograma**.\
Usaremos la columna de `Profundidad` y la función `hist`:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
ax.hist(lima["Profundidad"], bins=30, color="lightgreen", alpha=0.75, edgecolor="black")
ax.set_title("Distribución de profundidad de los sismos (en km)", fontsize=15)
ax.set_xticks([10*i for i in range(0, 17)])
plt.show()

Podemos observar que existen dos modas de profundidad de cada sismo: el primero a 50 km y el segundo a 100 km.\
Vamos a separar las profundidades de acuerdo a estos límites.\
Usaremos una función llamada `color_profundidad` para establecer un color de acuerdo a la profundidad de ocurrencia del sismo:

In [ ]:
def color_profundidad(profundidad):
    if profundidad <= 50:
        return "red"
    elif 50 < profundidad <= 100:
        return "gold"
    elif profundidad > 100:
        return "lightgreen"

Aplicando la función a la columna `Profundidad` y asignando el resultado a una nueva columna llamada `Color`:

In [ ]:
lima["Color"] = lima["Profundidad"].apply(color_profundidad)

Observamos los datos generales:

In [ ]:
lima.head()

Y ahora, graficaremos la distribución de profundidad usando un **diagrama de dispersión** y agregaremos los datos de color generados anteriormente:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={"aspect": 1})
ax.scatter(x=lima["Longitud"], y=lima["Latitud"], color=lima["Color"], s=5)

for ciudad, coordenada in ciudades.items():
    ax.scatter(coordenada[1], coordenada[0], c="b")
    ax.annotate(ciudad, coordenada[::-1], fontsize=14)

ax.set_title("Distribución de profundidad de sismos\n alrededor de Lima (Radio de 150 km)", fontsize=18)
ax.set_xlabel("Longitud (°)", fontsize=15)
ax.set_ylabel("Latitud (°)", fontsize=15)

ax.grid(linewidth=0.5, alpha=0.5)

for color, label in [("red", "Menor a 50 km"), ("gold", "Entre 50 y 100 km"), ("lightgreen", "Mayor a 100 km")]:
   ax.scatter([], [], color=color, label=label)

ax.legend(loc="upper right", fontsize=12)

plt.show()

***

<a id="parte-4"></a>

### <span style="color:lightgreen">**Visualización 3D con `Plotly`**</span>
***
Crearemos una función `cat_profundidad` que establezca un valor categórico de acuerdo a la profundidad de ocurrencia del sismo:

In [ ]:
def cat_profundidad(profundidad):
    if profundidad <= 50:
        return "Menor a 50 Km"
    elif 50 < profundidad <= 100:
        return "Entre 50 y 100 Km"
    else:
        return "Mayor a 100 Km"

Aplicamos la función en la columna `Profundidad` y asignamos el resultado a una nueva columna llamada `Categoría`:

In [ ]:
lima["Categoría"] = lima["Profundidad"].apply(cat_profundidad)

In [ ]:
# Mostramos 5 filas aleatorias
lima.sample(5)

Para terminar, visualizaremos la información en 3D usando `plotly`, una librería de visualización interactiva:

In [ ]:
import plotly.express as px

fig = px.scatter_3d(data_frame=lima, x="Longitud", y="Latitud", z=lima["Profundidad"]*-1, color=lima["Categoría"],
                    color_discrete_map={"Menor a 50 Km": "red", "Entre 50 y 100 Km": "orange", "Mayor a 100 Km": "blue"})

fig.update_layout(
    autosize=False,
    legend = dict(bgcolor="white", title="Profundidad", itemsizing="constant"),
    width=750,
    height=500,
    margin=dict(l=50,
                r=50,
                b=50,
                t=50,
                pad=4
               ),
    paper_bgcolor="LightSteelBlue",
    scene=dict(xaxis_title="Longitud",
               yaxis_title="Latitud",
               zaxis_title="Profundidad")
)

fig.update_traces(marker={"size": 1.5})

fig.show()

***